In [1]:
from google.colab import drive
drive.mount('/content/drive')
current_directory = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data'

from google.colab.patches import cv2_imshow

Mounted at /content/drive


In [2]:
import pandas as pd
import os

In [3]:
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Cloning into 'detectron2'...
remote: Enumerating objects: 15180, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 15180 (delta 110), reused 138 (delta 76), pack-reused 14979
Receiving objects: 100% (15180/15180), 6.22 MiB | 22.98 MiB/s, done.
Resolving deltas: 100% (10977/10977), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61406 sha256=0025541b565f23fd21919e7bdfd859ad5d52fedd2d6666943091cbc974af6c

In [4]:
using_colab = True
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-njjrgulf
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-njjrgulf
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36588 sha256=6e04023054ff6f8bfeb5fc7ec8b91c6a7dbc2b9c6675b89d6edee3ef7dc386cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-yv2y4bks/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [5]:
%pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git

  Cloning https://github.com/luca-medeiros/lang-segment-anything.git to /tmp/pip-req-build-s0xj_qul
  Running command git clone --filter=blob:none --quiet https://github.com/luca-medeiros/lang-segment-anything.git /tmp/pip-req-build-s0xj_qul
  Resolved https://github.com/luca-medeiros/lang-segment-anything.git to commit 2ebcd001b0ed9dac9645ed54c35def2bba3ddbd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/IDEA-Research/GroundingDINO.git to /tmp/pip-install-un_ar4bc/groundingdino_fa4e708be1f046eb92a2e525d6118d27
  Running command git clone --filter=blob:none --quiet https://github.com/IDEA-Research/GroundingDINO.git /tmp/pip-install-un_ar4bc/groundingdino_fa4e708be1f046eb92a2e525d6118d27
  Resolved https://github.com/IDEA-Research/GroundingDINO.git to commit 60d796825e1266e56f7e4e9e00e88de662b67bd3
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fac

In [6]:
# Install Ultralytics package
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 8.8 MB/s eta 0:00:00


In [9]:
import os
import random
import cv2
from ultralytics import YOLO
import torch
import numpy as np
import pandas as pd
#from scipy.stats import kruskal
#import scikit_posthocs as sp
import detectron2
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from lang_sam import LangSAM    #pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git
from PIL import Image
from lang_sam.utils import draw_image


In [78]:

def GetYOLOResult_Contour(results):

    # Create a list to store dictionaries with area and count for each result
    result_data = []

    for result in results:
        areas = []
        image_shape = result.orig_shape

        overall_mask = np.zeros((result.orig_shape[0], result.orig_shape[1]), dtype=np.uint8)
       # print(result)
        for j, mask in enumerate(result.masks.data):
        #   # Move the mask tensor to CPU if it's on a CUDA device
            mask_np = mask.detach().cpu().numpy() if isinstance(mask, torch.Tensor) else mask

        #   # Convert the mask to uint8 and resize it to match the image size
#            mask_np = cv2.resize((mask_np.astype( np.uint8) * 255), (result.orig_shape[1], result.orig_shape[0]))

            # Resize to original image size, Using INTER_NEAREST ensures resized mask remains binary with values 0 and 1, preserving its original nature.
            mask_np_resized = cv2.resize(mask_np, (result.orig_shape[1], result.orig_shape[0]), interpolation=cv2.INTER_NEAREST).astype(np.uint8)

            contours, _ = cv2.findContours(mask_np_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize the total area for this mask
            total_area = 0.0

            # Iterate through each contour and calculate its area
            for contour in contours:
                area = cv2.contourArea(contour)
                total_area += area

            #area_covered = np.sum(mask_np_resized)
            areas.append(total_area)

            # Combine the current mask with the overall mask using logical OR operation
            overall_mask = cv2.bitwise_or(overall_mask, mask_np_resized)

        result_data.append({'area': sum(areas), 'count': len(areas), 'overallMask': overall_mask})

    return result_data  # get count of fibres and area covered


def GetYOLOResult(results):

    # Create a list to store dictionaries with area and count for each result
    result_data = []

    for result in results:
        areas = []
        image_shape = result.orig_shape
        overall_mask = np.zeros((result.orig_shape[0], result.orig_shape[1]), dtype=np.uint8)
        for j, mask in enumerate(result.masks.data):
        #   # Move the mask tensor to CPU if it's on a CUDA device
            mask_np = mask.detach().cpu().numpy() if isinstance(mask, torch.Tensor) else mask

        #   # Convert the mask to uint8 and resize it to match the image size
#            mask_np = cv2.resize((mask_np.astype( np.uint8) * 255), (result.orig_shape[1], result.orig_shape[0]))

            # Resize to original image size, Using INTER_NEAREST ensures resized mask remains binary with values 0 and 1, preserving its original nature.
            mask_np_resized = cv2.resize(mask_np, (result.orig_shape[1], result.orig_shape[0]), interpolation=cv2.INTER_NEAREST).astype(np.uint8)

            area_covered = np.sum(mask_np_resized)
            areas.append(area_covered)

            # Combine the current mask with the overall mask using logical OR operation
            overall_mask = cv2.bitwise_or(overall_mask, mask_np_resized)

        result_data.append({'area': sum(areas), 'count': len(areas), 'overallMask': overall_mask})

    return result_data  # get count of fibres and area covered



def getMaskContours(mask):

 # Create a list to store dictionaries with area and count for each result
    result_data = []

    areas = []

    # Calculate the contour of the mask using cv2.findContours (only external contours)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize the total area for this mask
    total_area = 0.0

        # Iterate through each contour and calculate its area
    for contour in contours:
        area = cv2.contourArea(contour)
        total_area += area

    areas.append(total_area)

    result_data.append({'area': sum(areas), 'count': len(contours)})

    return result_data  # get count of fibres and area covered



def GetDetectronResult(results):

    # Create a list to store dictionaries with area and count for each result
    result_data = []

    # Access the predicted masks
    predicted_masks = results["instances"].pred_masks
    predicted_masks_np = predicted_masks.cpu().numpy().astype(np.uint8)

    image_size = results["instances"].image_size
    height, width = image_size

    overall_mask = np.zeros((height, width), dtype=np.uint8)
    areas = []
    for i, mask in enumerate(predicted_masks_np):

    # Calculate the contour of the mask using cv2.findContours (only external contours)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize the total area for this mask
        total_area = 0.0

        # Iterate through each contour and calculate its area
        for contour in contours:
            area = cv2.contourArea(contour)
            total_area += area

        areas.append(total_area)

        # Combine the current mask with the overall mask using logical OR operation
        overall_mask = cv2.bitwise_or(overall_mask, mask)

    result_data.append({'area': sum(areas), 'count': len(areas), 'overallMask': overall_mask})

    return result_data  # get count of fibres and area covered


def GetSAMresult(results):

    # Create a list to store dictionaries with area and count for each result
    result_data = []

    width = results[0]['crop_box'][2]          # Cropping is the same for all the image and so represents the size of the image
    height = results[0]['crop_box'][3]
    overall_mask = np.zeros((height, width), dtype=np.uint8)
    areas = []
#    print('results', results)

    for i in range(1, len(results)):   # Ignore the first finding because that always seemsto be the entire shape
        results[i]['segmentation']
        # Move the mask tensor to CPU if it's on a CUDA device
        #mask_np = results[i]['segmentation'].detach().cpu().numpy() if isinstance(results[i]['segmentation'], torch.Tensor) else results[i]['segmentation']

        mask_tensor = results[i]['segmentation']
        if isinstance(mask_tensor, torch.Tensor):
            mask_np = mask_tensor.detach().cpu().numpy()
        else:
            mask_np = mask_tensor
#        overall_mask[mask_np] = 1
        areas.append(results[i]['area'])

        # Combine the current mask with the overall mask using logical OR operation
        #overall_mask = cv2.bitwise_or(overall_mask, mask_np)
        # Find the maximum value in mask_np
        #max_value = np.max(mask_np)
        #print("Maximum value in mask_np:", max_value, 'loop', i)

        overall_mask[mask_np] = 1
    result_data.append({'area': sum(areas), 'count': len(results), 'overallMask': overall_mask})
    return result_data  # get count of fibres and area covered



def getSAMLang_Result(result):

    result_data = []
    mask_np = result.detach().cpu().numpy().astype(np.uint8) if isinstance(result, torch.Tensor) else result

   # Access the predicted masks

    image_size = mask_np.shape

    height = image_size[1]
    width = image_size[2]

    overall_mask = np.zeros((height, width), dtype=np.uint8)
    areas = []
    for i, mask in enumerate(mask_np):

    # Calculate the contour of the mask using cv2.findContours (only external contours)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize the total area for this mask
        total_area = 0.0

        # Iterate through each contour and calculate its area
        for contour in contours:
            area = cv2.contourArea(contour)
            total_area += area

        areas.append(total_area)

        # Combine the current mask with the overall mask using logical OR operation
        overall_mask = cv2.bitwise_or(overall_mask, mask)

    result_data.append({'area': sum(areas), 'count': len(areas), 'overallMask': overall_mask})

    return result_data  # get count of fibres and area covered

def show_anns(anns):
    # for use when displaying segment anything mask
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

def prep_mask_image(anns):
    if len(anns) == 0:
        return None
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    img = (img * 255).astype(np.uint8)  # Convert to uint8 range (0-255)
    return Image.fromarray(img)




def calculate_iou(groundTruth, predicted):

   # print(' calc iou', np.max(predicted))

    intersection = np.logical_and(groundTruth, predicted).sum()
    union = np.logical_or(groundTruth, predicted).sum()
    iou = intersection / union

    # Next am curious as to how much of the predicted objects are contained within the mask objects
    # so am checking how much of the intersenting is in the predicted. This is because I think the ground truth masks
    # might be a bit bigger than the fibres so it might score lower on IOU but still be an accurate prediction.
    int_predicted_intersection = intersection / predicted.sum()

    dice_coefficient = 2 * np.sum(intersection) / (np.sum(groundTruth) + np.sum(predicted))
    return iou , dice_coefficient, int_predicted_intersection



In [42]:

   # train on the GPU or on the CPU, if a GPU is not available - CPU will be very slow
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
# Source Data

fibre_images = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Set/images/test'
fibre_masks = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Set/masks/test'
img_files = os.listdir(fibre_images) # Get list of files in the directory

##########################################################################################
# YOLO Model
##########################################################################################
# preTrainedYOLO = r'/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Models/YoloLTrain1.pt'
# # Load a model
# YOLOmodel = YOLO(preTrainedYOLO)  # pretrained YOLOv8n model

# ##########################################################################################
# # Detectron
# ##########################################################################################
# cfg = get_cfg()

# #cfg.MODEL.DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# cfg.MODEL.DEVICE = device.type
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# ###
# ####### Change this
# #cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
# #cfg.MODEL.WEIGHTS = r'C:\Users\dezos\Documents\Fibres\FibreAnalysis\Step1_DataCreation\Step4_Evaluate\Detectron2_Trained_Model.pth'
# cfg.MODEL.WEIGHTS = r'/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Models/Detectron2_Trained_Model.pth'
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # Model was trained on one class so need to specify that here


##########################################################################################
# Segment Anything
##########################################################################################

sam_checkpoint = r"/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Models/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = device

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

sam_mask_generator = SamAutomaticMaskGenerator(sam)

###########################################################################################
# Language Segment anything
###########################################################################################

samL = LangSAM()

##
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
#Detect2Predictor = DefaultPredictor(cfg)

cuda
final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from /root/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])


In [77]:
dataHolder = []
header_names = ['Image', 'YOLO_iou_value', 'YOLO_diceCoef' , 'YOLO_predIOU', 'Detect_iou_value', 'Detect_diceCoef', 'Detect_predIOU',
                'SAM_iou_value', 'SAM_diceCoef', 'SAM_predIOU', 'SAMlang_iou_value', 'SAMlang_diceCoef', 'SAMlang_predIOU',
                'SAM Area', 'SAM Count' , 'SAM Lang area', 'SAM Lang count', 'Ground Truth area', 'Ground Truth count' ]

# Assuming 'counter' is defined elsewhere in your code

path = r'/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/EvalResults'

counter = 0
#for d in random.sample(img_files, 2):
for i, d in enumerate(random.sample(img_files, 2)):
    try:
      print('loop', i)
      fullPath = os.path.join(fibre_images, d)
      mask_path = os.path.join(fibre_masks, f"mask{d[5:]}")  # Change filename from image_ to mask _ to retrieve mask
      print(fullPath)  # Print File name
      print(mask_path)  # Print File name
      image = cv2.imread(fullPath)  # Load your input image
      ground_truth_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

      ground_mask_details = getMaskContours(ground_truth_mask)

#      YOLOresult = GetYOLOResult_Contour(YOLOmodel(fullPath))   # get area and count for YOLO

#      DetectronResult = GetDetectronResult(Detect2Predictor(image))
      #print('detectron results', DetectronResult)
      SAMmasks = sam_mask_generator.generate(image)
      SAMresults = GetSAMresult(SAMmasks)

      # Language

      mask_image = prep_mask_image(SAMmasks)
      grayscale_image = mask_image.convert("L")
      # Create a new RGB image with grayscale values in all channels
      rgb_bw_image = Image.new("RGB", grayscale_image.size)
      rgb_bw_image.paste(grayscale_image)
      text_prompt = "line"

      SAMLmasks, SAMLboxes, SAMLlabels, SAMLlogits = samL.predict(rgb_bw_image, text_prompt, box_threshold=0.20, text_threshold=0.24)


      SAMlang_results = getSAMLang_Result(SAMLmasks)
      #Calculate IoU and Dice coefficient
  #    YOLO_iou_value, YOLO_diceCoef , YOLO_predIOU = calculate_iou(ground_truth_mask, YOLOresult[-1]["overallMask"])
  #    Detect_iou_value, Detect_diceCoef, Detect_predIOU = calculate_iou(ground_truth_mask, DetectronResult[-1]["overallMask"])

      SAM_iou_value, SAM_diceCoef, SAM_predIOU = calculate_iou(ground_truth_mask, SAMresults[-1]["overallMask"])

      SAMlang_iou_value, SAMlang_diceCoef, SAMlang_predIOU = calculate_iou(ground_truth_mask, SAMlang_results[-1]["overallMask"])


      #Prepare rows for dataframe later. See this link as to why to create a list first (much faster)  https://stackoverflow.com/questions/10715965/create-a-pandas-dataframe-by-appending-one-row-at-a-time
   #   dataHolder.append([d, YOLO_iou_value, YOLO_diceCoef , YOLO_predIOU, Detect_iou_value, Detect_diceCoef, Detect_predIOU,
   #                         SAM_iou_value, SAM_diceCoef, SAM_predIOU, SAMlang_iou_value, SAMlang_diceCoef, SAMlang_predIOU ])

      dataHolder.append([d, '0', '0' , '0', '0', '0', '0',
                         SAM_iou_value, SAM_diceCoef, SAM_predIOU, SAMlang_iou_value, SAMlang_diceCoef, SAMlang_predIOU,
                         SAMresults[-1]["area"], SAMresults[-1]["count"], SAMlang_results[-1]["area"], SAMlang_results[-1]["count"],
                         ground_mask_details[-1]["area"], ground_mask_details[-1]["count"]])



    # Increment the counter
      counter += 1

      # Check if the counter is a multiple of 10
      if counter % 5 == 0:
          # Create a DataFrame from the list
          df = pd.DataFrame(dataHolder,  columns=header_names)

          filename = f'IOUResults_{counter}.csv'
          fullPath = os.path.join(path, filename)
          df.to_csv(fullPath, index=False)

    except Exception as e:
    # Code to handle the error
      print("An error occurred:", e, fullPath )
    # Additional error handling or fallback actions can be added here


# After the loop ends, check if there are any remaining data in dataHolder
# Save it to a CSV file if needed
if dataHolder:
    df = pd.DataFrame(dataHolder,  columns=header_names)
    filename = f'IOUResults_{counter}.csv'
    fullPath = os.path.join(path, filename)
    df.to_csv(fullPath, index=False)



loop 0
/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Set/images/test/image_2023-07-24_19-40-35-581733_1.png
/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Set/masks/test/mask_2023-07-24_19-40-35-581733_1.png
Maximum value in mask_np: True loop 1
Maximum value in mask_np: True loop 2
Maximum value in mask_np: True loop 3


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:874: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


loop 1
/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Set/images/test/image_2023-07-24_19-41-50-577683_2.png
/content/drive/MyDrive/Colab Notebooks/FibreAnalysis/Data/Prepared/Set/masks/test/mask_2023-07-24_19-41-50-577683_2.png
Maximum value in mask_np: True loop 1
Maximum value in mask_np: True loop 2
Maximum value in mask_np: True loop 3
Maximum value in mask_np: True loop 4
Maximum value in mask_np: True loop 5
Maximum value in mask_np: True loop 6
Maximum value in mask_np: True loop 7
Maximum value in mask_np: True loop 8
Maximum value in mask_np: True loop 9
Maximum value in mask_np: True loop 10


In [35]:
torch.cuda.empty_cache()

In [49]:
# Print the current GPU memory allocated by tensors in bytes
allocated_memory = torch.cuda.memory_allocated()
print(f"Currently allocated GPU memory: {allocated_memory / 1024**2:.2f} MB")

# Print a summary of GPU memory usage and list of tensors
summary = torch.cuda.memory_summary()
print(summary)

Currently allocated GPU memory: 6243.55 MB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 33           |        cudaMalloc retries: 34        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   6243 MiB |  12550 MiB |   9329 GiB |   9323 GiB |
|       from large pool |   6068 MiB |  12362 MiB |   9262 GiB |   9256 GiB |
|       from small pool |    174 MiB |    199 MiB |     66 GiB |     66 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   6243 MiB |  12550 MiB |   9329 GiB |   9323 GiB |
|       from large po